In [1]:
import copy, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Multiple Linear Regression
We're going to use a small data set to apply what we've learnt about multiple linear regression. Each row is an observation of a student's Psych test results for three term exams and their finals. We'll be using three features: Exams 1, 2, and 3 to try and the student's final exam result will be the target variable

In [26]:
df = pd.read_csv("psych_test_results.csv")
df.tail()

,EXAM1,EXAM2,EXAM3,FINAL
20,82,86,90,177
21,86,82,89,175
22,78,83,85,175
23,76,83,71,149
24,96,93,95,192


In [3]:
X = np.array(df.iloc[:,0:3])
y = np.array(df.iloc[:,3])
train_size = int(X.shape[0]*0.8)
print(train_size)

20


In [4]:
# Taking an 80:20 test:train split
X_train = X[0:train_size,:]
y_train = y[0:train_size]
X_test = X[train_size:,:]
y_test = y[train_size:]

In [5]:
print(f"X_train shape: {X_train.shape} and\ny_train shape: {y_train.shape}")

X_train shape: (20, 3) and
y_train shape: (20,)


In [6]:
b_init = 0
w_init = np.zeros(3,)
print(w_init.shape)

(3,)


We're carefully keeping track of the shape of our data and ensuring that our model's weights (w) have the correct shape for the number of features. 

The model's prediction with multiple variables is given by the linear model:

$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
or in vector notation:
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 
where $\cdot$ is a vector `dot product`

In [7]:
def compute_cost(X, y, w, b):
    """
    
    computes cost of model
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    
    m = X.shape[0] # number of examples
    cost = 0.0 # initializing cost
    for i in range(m):
        f_wb_i = np.dot(X[i], w) + b   # dot product of the ith example. A scalar
        cost = cost + (f_wb_i - y[i])**2 # summing cost
    cost = cost/(2*m)
    return cost

In [8]:
def compute_gradient(X, y, w, b):
    """
    Computes gradient for linear regression
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters
      b (scalar)       : model parameter
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w
      dj_db (scalar       : The gradient of the cost w.r.t. the parameter b
    """
    m,n = X.shape #m examples, n features
    # initialize gradient w.r.t paremeters 
    dj_dw = np.zeros((n,)) # vector (array) of zeros of size n
    dj_db = 0.0
    
    for i in range(m):
        err = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err * X[i, j]
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m
    
    return dj_db, dj_dw    

In [9]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters):
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))  : Data, m examples with n features
      y (ndarray (m,))   : target values 
      w_in (ndarray (n,)): initial model parameters
      b_in(scalar)       : initial model paramter
      cost_function      : function to compute cost
      gradient_function  : function to compute gradient
      alpha (float)      : learning rate
      num_iters (int)    : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : updated values of parameters 
      b (scalar)       : updated value of parameter
    """
    
    w = copy.deepcopy(w_in) # Avoid altering w_in
    b = b_in
    
    for i in range(num_iters):
        
        dj_db, dj_dw = gradient_function(X, y, w, b) # this stores the respective gradients so we can update w and b simultaneously i.e. 
        
        # update parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw
        b = b - alpha * dj_db # dj_db isn't altered by the new w value
        
    return w, b

In [10]:
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f'cost for parameters w and b initialized at 0: {cost}')

cost for parameters w and b initialized at 0: 12966.375


In [11]:
# carrying out multiple linear regression via gradient descent
alpha = 0.0001  # step size
w, b = gradient_descent(X_train, y_train, w_init, b_init, compute_cost, compute_gradient, alpha, 10000)
print(f"For our model, the parameters w are {w} and b is {round(b, 3)}")

In [29]:
test_cost = math.sqrt(compute_cost(X_test, y_test, w, b)*2) # rms error
print(f"Our model has a root mean squared error of {round(test_cost, 3)} against the test data set")

Our model has a root mean squared error of 3.974 against the test data set


In [30]:
# Performance of our model against the test set. Just something to eyeball
our_predictions = np.zeros(5,)
for i in range(X_test.shape[0]):
    y = np.dot(w, X_test[0 + i]) + b
    our_predictions[i] = y # storing our test prediction in an array
    print(f"Our predicted value for observation {21+i} is: {round(y, 1)} with actual value {y_test[0 + i]}")

Our predicted value for observation 21 is: 175.3 with actual value 177
Our predicted value for observation 22 is: 173.9 with actual value 175
Our predicted value for observation 23 is: 166.8 with actual value 175
Our predicted value for observation 24 is: 151.1 with actual value 149
Our predicted value for observation 25 is: 190.4 with actual value 192


### Using Scikit-learn
We'll implement a model using sk-learn just to compare performance.

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)
sk_predictions = model.predict(X_test)

In [16]:
for i in range(X_test.shape[0]):
    print(f"SKlearns predicted value for observation {21+i} is: {round(sk_predictions[0+i], 1)} with actual value {y_test[0 + i]}")

SKlearns predicted value for observation 21 is: 175.6 with actual value 177
SKlearns predicted value for observation 22 is: 174.2 with actual value 175
SKlearns predicted value for observation 23 is: 166.9 with actual value 175
SKlearns predicted value for observation 24 is: 150.8 with actual value 149
SKlearns predicted value for observation 25 is: 190.8 with actual value 192


In [17]:
our_mse= mean_squared_error (y_test, our_predictions)
sk_mse = mean_squared_error(y_test, sk_predictions)
print(f"The mse for our model: {round(our_mse,3)} vs sk's mse: {round(sk_mse,3)}")

The mse for our model: 15.818 vs sk's mse: 14.732
